# Optimization Techniques in Machine Learning

Objective: This assignment aims to explore implementation or Machine Learning Models with regularization, optimization and Error analysis  techniques used in machine learning to improve models' performance, convergence speed, and efficiency..

A Notebook detailing the following

* Project name
* Clear out puts from cells






**Instructions**

1. Acquire a dataset suitable for ML tasks as per your proposal.
2. Implement a simple machine learning model based on neural networks on the chosen dataset without any defined optimization techniques. (Check instructions)
3. Implement and compare the model's performance after applying 3 to 4 disntict combinations regularization and optimization techniques.
4. Discuss the results on the README file.
5. Make predictions using test data
7. Implement error analysis techniques and ensure there is: F1-Score, Recall, Precision, RUC a confusion matrix using plotting libraries (not verbose)

Submit notebook to github repo




# Case Study and Implementation




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2


ModuleNotFoundError: No module named 'pandas'

# The Dataset
> ***Brief Description:***
State the Problem and A short Description of the data


In [ ]:
df = pd.read_csv('Maternal Health Risk Data Set.csv')


In [30]:
print("Data Preview:")
display(df.head())

Data Preview:


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [31]:
print(df['RiskLevel'].value_counts())

RiskLevel
low risk     406
mid risk     336
high risk    272
Name: count, dtype: int64


In [32]:
label_enc = LabelEncoder()
df['RiskLevel'] = label_enc.fit_transform(df['RiskLevel'])

In [33]:
X = df.drop('RiskLevel', axis=1)
y = df['RiskLevel']

In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [36]:
print("\\nData Description:")
display(df.describe())

print("\\nClass Distribution:")
print(df['RiskLevel'].value_counts())

\nData Description:


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
count,1014.000000,1014.000000,1014.000000,1014.000000,1014.000000,1014.000000,1014.000000
mean,29.871795,113.198225,76.460552,8.725986,98.665089,74.301775,1.063116
std,13.474386,18.403913,13.885796,3.293532,1.371384,8.088702,0.772146
min,10.000000,70.000000,49.000000,6.000000,98.000000,7.000000,0.000000
25%,19.000000,100.000000,65.000000,6.900000,98.000000,70.000000,0.000000
50%,26.000000,120.000000,80.000000,7.500000,98.000000,76.000000,1.000000
75%,39.000000,120.000000,90.000000,8.000000,98.000000,80.000000,2.000000
max,70.000000,160.000000,100.000000,19.000000,103.000000,90.000000,2.000000


\nClass Distribution:
RiskLevel
1    406
2    336
0    272
Name: count, dtype: int64


In [37]:
label_enc = LabelEncoder()
df['RiskLevel'] = label_enc.fit_transform(df['RiskLevel'])

X = df.drop('RiskLevel', axis=1)
y = df['RiskLevel']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#SECTION 1: Model Architecture:



```
TODO: Insert an image with the Model architecture here.Replace the image Below
```
> <img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*v1ohAG82xmU6WGsG2hoE8g.png" alt="?" style="width:25px"/>




Classical ML Model (Logistic Regression with Hyperparameter Tuning)

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import joblib

log_model = LogisticRegression(C=0.1, penalty='l2', solver='liblinear')
log_model.fit(X_train, y_train)

y_pred_log = log_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred_log))



Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.73      0.81        41
           1       0.60      0.95      0.73        61
           2       0.52      0.24      0.32        51

    accuracy                           0.65       153
   macro avg       0.68      0.64      0.62       153
weighted avg       0.66      0.65      0.62       153



Define Neural Network Model (Modular)

In [40]:
def define_model(optimizer_name='adam', reg_type=None, reg_value=0.01, dropout=0.0, learning_rate=0.001,
                 early_stopping=False, epochs=50, num_layers=2, save_path=None):
    model = Sequential()

    # Regularizer
    reg = l1(reg_value) if reg_type == 'l1' else l2(reg_value) if reg_type == 'l2' else None

    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=reg))
    if dropout > 0.0:
        model.add(Dropout(dropout))

    for _ in range(num_layers - 1):
        model.add(Dense(32, activation='relu', kernel_regularizer=reg))

    model.add(Dense(3, activation='softmax'))

    # Optimizer
    optimizer = Adam(learning_rate=learning_rate) if optimizer_name == 'adam' else RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    callbacks = []
    if early_stopping:
        callbacks.append(EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True))

    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=epochs, batch_size=32, verbose=0, callbacks=callbacks)

    if save_path:
        model.save(save_path)

    return model, history


Plot Loss Curve

In [41]:
def loss_curve_plot(history, title='Loss Curve'):
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()


Evaluate Model

In [42]:
def evaluate_model(model, X_eval, y_eval):
    y_probs = model.predict(X_eval)
    y_pred = np.argmax(y_probs, axis=1)

    cm = confusion_matrix(y_eval, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    print(classification_report(y_eval, y_pred))
    print("ROC AUC Score:", roc_auc_score(pd.get_dummies(y_eval), y_probs))


 Train Neural Network Instances

 Instance 1: Simple NN (No optimization)

In [44]:
model1, history1 = define_model(epochs=30, save_path="saved_models/simple_nn_instance1.keras")
loss_curve_plot(history1, "Instance 1 Loss Curve")
evaluate_model(model1, X_test, y_test)


FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/simple_nn_instance1.keras'

Instance 2: With Dropout, L2 Regularization, RMSProp

In [ ]:
model2, history2 = define_model(optimizer_name='rmsprop', dropout=0.3, reg_type='l2',
                                learning_rate=0.001, epochs=50, save_path="saved_models/optimized_nn_instance2.keras")
loss_curve_plot(history2, "Instance 2 Loss Curve")
evaluate_model(model2, X_test, y_test)


 Instance 3: Adam, Early Stopping, L1 Regularization

In [ ]:
model3, history3 = define_model(optimizer_name='adam', reg_type='l1', early_stopping=True,
                                learning_rate=0.0005, dropout=0.2, epochs=100,
                                save_path="saved_models/optimized_nn_instance3.keras")
loss_curve_plot(history3, "Instance 3 Loss Curve")
evaluate_model(model3, X_test, y_test)


 Instance 4: Increased Layers, L2, Early Stopping

In [ ]:
model4, history4 = define_model(optimizer_name='adam', reg_type='l2', early_stopping=True,
                                learning_rate=0.0001, dropout=0.25, num_layers=3,
                                epochs=100, save_path="saved_models/optimized_nn_instance4.keras")
loss_curve_plot(history4, "Instance 4 Loss Curve")
evaluate_model(model4, X_test, y_test)


Load and Predict from Saved Model

In [13]:
def make_predictions(model_path, X_input):
    model = load_model(model_path)
    pred_probs = model.predict(X_input)
    return np.argmax(pred_probs, axis=1)

predictions = make_predictions("saved_models/optimized_nn_instance3.keras", X_test)
print(predictions[:10])


NameError: name 'notebook_cells' is not defined

#Task: Define a function that creates models without and With specified Optimization techniques


In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizer
from tensorflow.keras.callbacks import EarlyStopping


def define_model(optimization: string, regularization_datatype, early_stopping: bool, dropout: float, learning_rate: float):
  model= None
  model.add(None)
  #TO DO: Add more layers as per architecture
  model.add(None) # Last Layer
  model.compile(optimizer = optimizerNone)
  model.fit(None)
  return model

# Task: Print out the Final Model Accuracy and plot the Loss curve

In [ ]:
def loss_curve_plot(None):
  epochs = None
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title('Training and Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

# SECTION 2: Optimization and Regularization Combinations
At this point you should now create models that combine various optimization techniques
As done before make sure to plot out the loss curve and the accuracy and loss in verbose

In [ ]:
#TODO:
model_2 = define_model('Adam', None)
loss_curve_plot(model_2):
#print out confusion matrix and error analysis metrics after the cell

In [ ]:
#TODO:
model_3 = define_model('RMSPop',None)
loss_curve_plot(model_3):
#print out confusion matrix and error analysis metrics after the cell

In [ ]:
#TODO:
model_4 = define_model(None)
loss_curve_plot(model_4):
#print out confusion matrix and error analysis metrics after the cell

#Task: Make Predictions using the best saved model


Create a confusion Matrix and F1 score for both Models. Ensure outputs for the cells are visible

Finally, Make predictions using the best model. By the time you get to this cell you may realise at some point you needed to save the model so that you cal load it later

In [ ]:
def make_predictions(model_path, X):

    # Load the model
    model = load_model(None)
    # Make predictions
    predictions = None
    # Convert probabilities to binary labels (0 or 1)

    return predictions

#Modify the code appropriately

In [ ]:
model_path = None
make_predictions(None)

Congratulations!!
